<h1><center>CheXNet fine-tuned + model Interpretation</center></h1>
<center><img src="https://images.unsplash.com/photo-1584555684040-bad07f46a21f?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=721&q=80" width="40%"></center>

**CheXNet** [[1]](https://arxiv.org/pdf/1711.05225.pdf) is a 121 layer **DenseNet** developed by Stanford researchers that can detect pneumonia from chest X-rays at a level exceeding practicing radiologists. The weights of the model are uploaded into this notebook and used to train on our data to classify normal vs opacity (typical, atypical, indeterminate) cases. Contrast Limited Adaptive Histogram Equalization (**CLAHE**) is used for preprocessing and some augmentation techniques are applied. For interpretability, **GRAD-CAM** is used to see if the model is paying attention to the opacities (comparing to the groundtruth bounding boxes).

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import cv2
import os
from skimage import exposure
import matplotlib
matplotlib.rcParams.update({'font.size': 16})
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.math import confusion_matrix
from sklearn.metrics import accuracy_score
from seaborn import heatmap
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from ast import literal_eval
from matplotlib.patches import Rectangle

In [ ]:
df_image = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
df_study = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
df_study['id'] = df_study['id'].str.replace('_study',"")
df_study.rename({'id': 'StudyInstanceUID'},axis=1, inplace=True)
df_train = df_image.merge(df_study, on='StudyInstanceUID')
df_train.loc[df_train['Negative for Pneumonia']==1, 'study_label'] = 'negative'
df_train.loc[df_train['Typical Appearance']==1, 'study_label'] = 'typical'
df_train.loc[df_train['Indeterminate Appearance']==1, 'study_label'] = 'indeterminate'
df_train.loc[df_train['Atypical Appearance']==1, 'study_label'] = 'atypical'
df_train.drop(['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], axis=1, inplace=True)
df_train['id'] = df_train['id'].str.replace('_image', '.jpg')
df_train['image_label'] = df_train['label'].str.split().apply(lambda x : x[0])
df_size = pd.read_csv('../input/covid-jpg-512/size.csv')
df_train = df_train.merge(df_size, on='id')
df_train.head(3)

## Preprocessing

Simply using Contrast Limited Adaptive Histogram Equalization (CLAHE) after passing them to the generator

In [ ]:
train_dir = '../input/covid-jpg-512/train'

def preprocess_image(img):
    equ_img = exposure.equalize_adapthist(img/255, clip_limit=0.05, kernel_size=24)
    return equ_img

df_opa = df_train[df_train['image_label']=='opacity'].reset_index()
fig, axs = plt.subplots(5, 2, figsize=(10,20))
fig.subplots_adjust(hspace=.2, wspace=.2)
n=5
for i in range(n):
    img = cv2.imread(os.path.join(train_dir, df_opa['id'][i]))
    img_proc = preprocess_image(img)
    axs[i, 0].imshow(img)
    axs[i, 1].imshow(img_proc)
    axs[i, 0].axis('off')
    axs[i, 1].axis('off')
    boxes = literal_eval(df_opa['boxes'][i])
    for box in boxes:
        axs[i, 0].add_patch(Rectangle((box['x']*(512/df_opa['dim1'][i]), box['y']*(512/df_opa['dim0'][i])), box['width']*(512/df_opa['dim1'][i]), box['height']*(512/df_opa['dim0'][i]), fill=0, color='y', linewidth=3))
        axs[i, 0].set_title(df_opa['study_label'][i])
        axs[i, 1].add_patch(Rectangle((box['x']*(512/df_opa['dim1'][i]), box['y']*(512/df_opa['dim0'][i])), box['width']*(512/df_opa['dim1'][i]), box['height']*(512/df_opa['dim0'][i]), fill=0, color='r', linewidth=3))
        axs[i, 1].set_title('After CLAHE')
    
plt.show()

## ImageGenerators and Augmentations

In [ ]:
img_size = 224
batch_size = 16

image_generator = ImageDataGenerator(
        validation_split=0.2,
        horizontal_flip = True,
        zoom_range = 0.15,
        brightness_range = [0.8, 1.2],
        fill_mode='nearest',
        preprocessing_function=preprocess_image
)

image_generator_valid = ImageDataGenerator(validation_split=0.2,preprocessing_function=preprocess_image)

train_generator = image_generator.flow_from_dataframe(
        dataframe = df_train,
        directory='../input/covid-jpg-512/train',
        x_col = 'id',
        y_col =  'image_label',  
        target_size=(img_size, img_size),
        batch_size=batch_size,
        subset='training', seed = 23) 

valid_generator=image_generator_valid.flow_from_dataframe(
    dataframe = df_train,
    directory='../input/covid-jpg-512/train',
    x_col = 'id',
    y_col = 'image_label',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation', shuffle=False, seed=23) 


In [ ]:
for j in range(4):
    aug_images = [train_generator[0][0][j] for i in range(5)]
    fig, axes = plt.subplots(1, 5, figsize=(24,24))
    axes = axes.flatten()
    for img, ax in zip(aug_images, axes):
        ax.imshow(img)
        ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
def load_process(img, img_size):
    img = load_img(img, target_size = (img_size, img_size))
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = preprocess_image(img)
    return img

## Architecture

In [ ]:
chex_weights_path = '../input/chexnet-weights/brucechou1983_CheXNet_Keras_0.3.0_weights.h5'

pre_model = DenseNet121(weights=None,
                                include_top=False,
                                input_shape=(img_size,img_size,3)
                               )
out = Dense(14, activation='sigmoid')(pre_model.output)
pre_model = Model(inputs=pre_model.input, outputs=out) 
pre_model.load_weights(chex_weights_path)
pre_model.trainable = False
x = pre_model.layers[-2].output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.1)(x)
output = Dense(2, activation='softmax')(x)
model = Model(pre_model.input, output)

In [ ]:
model.compile(Adam(lr=1e-3),loss='binary_crossentropy',metrics='accuracy')

In [ ]:
rlr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.2, patience = 2, verbose = 1, 
                                min_delta = 1e-4, min_lr = 1e-4, mode = 'max')
es = EarlyStopping(monitor = 'val_accuracy', min_delta = 1e-4, patience = 5, mode = 'max', 
                    restore_best_weights = True, verbose = 1)

ckp = ModelCheckpoint('model.h5',monitor = 'val_accuracy',
                      verbose = 0, save_best_only = True, mode = 'max')

history = model.fit(
      train_generator,
      epochs=20,
      validation_data=valid_generator,
      callbacks=[es,rlr, ckp],
      verbose=1)

## Fine-tuning

In [ ]:
pre_model.trainable = True

model.compile(Adam(lr=1e-5),loss='binary_crossentropy',metrics='accuracy')

In [ ]:
rlr2 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.1, patience = 2, verbose = 1, 
                                min_delta = 1e-4, min_lr = 1e-7, mode = 'max')

es2 = EarlyStopping(monitor = 'val_accuracy', min_delta = 1e-4, patience = 7, mode = 'max', 
                    restore_best_weights = True, verbose = 1)

history2 = model.fit(
      train_generator,
      epochs=40,
      validation_data=valid_generator,
      callbacks=[es2,rlr2, ckp],
      verbose=1)

K.clear_session()

## Model performance

In [ ]:
actual =  valid_generator.labels
preds = np.argmax(model.predict(valid_generator), axis=1)
cfmx = confusion_matrix(actual, preds)
acc = accuracy_score(actual, preds)

In [ ]:
print ('Test Accuracy:', acc )
heatmap(cfmx, annot=True, cmap='plasma',
        xticklabels=['Normal','Opacity'],fmt='.0f', yticklabels=['Normal', 'Opacity'])
plt.show()

In [ ]:
hist = pd.DataFrame(history.history)
fig, (ax1, ax2) = plt.subplots(figsize=(12,12),nrows=2, ncols=1)
hist['loss'].plot(ax=ax1,c='k',label='training loss')
hist['val_loss'].plot(ax=ax1,c='r',linestyle='--', label='validation loss')
ax1.legend()
hist['accuracy'].plot(ax=ax2,c='k',label='training accuracy')
hist['val_accuracy'].plot(ax=ax2,c='r',linestyle='--',label='validation accuracy')
ax2.legend()
plt.show()

In [ ]:
hist = pd.DataFrame(history2.history)
fig, (ax1, ax2) = plt.subplots(figsize=(12,12),nrows=2, ncols=1)
hist['loss'].plot(ax=ax1,c='k',label='training loss')
hist['val_loss'].plot(ax=ax1,c='r',linestyle='--', label='validation loss')
ax1.legend()
hist['accuracy'].plot(ax=ax2,c='k',label='training accuracy')
hist['val_accuracy'].plot(ax=ax2,c='r',linestyle='--',label='validation accuracy')
ax2.legend()
plt.show()

## Model Interpretation 

In [ ]:
def grad_cam(input_image, model, layer_name):

    desired_layer = model.get_layer(layer_name)
    grad_model = Model(model.inputs, [desired_layer.output, model.output])

    with tf.GradientTape() as tape:
        layer_output, preds = grad_model(input_image)
        ix = (np.argsort(preds, axis=1)[:, -1]).item()
        output_idx = preds[:, ix]

    gradient = tape.gradient(output_idx, layer_output)
    alpha_kc = np.mean(gradient, axis=(0,1,2))
    L_gradCam = tf.nn.relu(np.dot(layer_output, alpha_kc)[0])
    L_gradCam = (L_gradCam - np.min(L_gradCam)) / (np.max(L_gradCam) - np.min(L_gradCam)) 
    return L_gradCam.numpy()

In [ ]:
def blend(img_path, gradCam_img, alpha, colormap = cv2.COLORMAP_JET):
    origin_img = img_to_array(load_img(img_path))
    gradCam_resized = cv2.resize(gradCam_img, (origin_img.shape[1], origin_img.shape[0]), interpolation = cv2.INTER_LINEAR)
    heatmap  = cv2.applyColorMap(np.uint8(gradCam_resized*255), colormap)
    superimposed_image = cv2.cvtColor(origin_img.astype('uint8'), cv2.COLOR_RGB2BGR) + heatmap * alpha
    return heatmap, superimposed_image

In [ ]:
def plot_results(model, gen, label=0):
    n = 50
    fig, axs = plt.subplots(10, 5, figsize=(20,60))
    fig.subplots_adjust(hspace=.5, wspace=.1)
    axs = axs.ravel()
    gen.next()
    classes = list(gen.class_indices.keys()) 
    if label==0:
        idx = np.array(np.where(np.array(gen.labels) ==0)).ravel()
    else:
        idx = np.array(np.where(np.array(gen.labels) ==1)).ravel()
   
    layer_name = 'bn'
    for i in range(n):
        sample_img_path = os.path.join(train_dir, df_train['id'][idx[i]])
        img = load_process(sample_img_path, img_size)
        pred = model.predict(img)
        grad_cam_img = grad_cam(img, model, layer_name)
        heatmap_img, result_img = blend(sample_img_path, grad_cam_img, 0.5)
        axs[i].imshow(result_img[:,:,::-1]/255)
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        if type(df_train['boxes'][idx[i]])==str:
            boxes = literal_eval(df_train['boxes'][idx[i]])
            for box in boxes:
                axs[i].add_patch(Rectangle((box['x']*(512/df_train['dim1'][idx[i]]), box['y']*(512/df_train['dim0'][idx[i]])), box['width']*(512/df_train['dim1'][idx[i]]), box['height']*(512/df_train['dim0'][idx[i]]), fill=0, color='y', linewidth=2))
                axs[i].set_title(f"{df_train['study_label'][idx[i]]}, {df_train['image_label'][idx[i]]}")
        else:
            axs[i].set_title(df_train['study_label'][idx[i]])
        
        axs[i].set_xlabel(f"{classes[np.argmax(pred)]}, {round(pred[0][np.argmax(pred)]*100, 2)}%")

In [ ]:
plot_results(model, valid_generator,label=0)

In [ ]:
plot_results(model, valid_generator,label=1)